In [20]:
'''
@ Author: Murdock
@ E-mail: mazhirenshnu@163.com
@ Date: 2022.10.14 Fri 
@ Title: 爬虫整理
@ Coding: utf-8

## Request

In [1]:
# 加载库
import pandas as pd
import numpy as np
import os

# 爬虫相关库
import re
import time
import requests
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

### Request 爬取陕西基金

In [3]:
# Request
# 获取目标网页链接
url = 'https://kjt.shaanxi.gov.cn/kjxmk/s1/y2/g1/252276.html'

# 模拟浏览器进行查找
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.33',
           'Content-Type': 'text/html',
           'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6,ja;q=0.5'}

# 进行网页链接访问
ret = Request(url, headers=headers)

# 打开网页链接
res = urlopen(ret)

# 读取网页内容
contents = res.read()

# 将网页内容整理至soup中
soup = BeautifulSoup(contents, "html.parser")

In [6]:
# 提取tag
# soup.find_all
tag = soup.find_all('div',class_="leaderinfo")[0]

# 整理tag
tag_title = tag.find_all('td', class_="td_title2")      # tag.title
tag_content = tag.find_all('td', class_="td_content2")  # tag.content

In [7]:
# 提取tag.title
title = []
for i in range(len(tag_title)):
    temp = str(tag_title [i])
    temp_findall = re.findall('>.+<',temp)[0] # 匹配>与<之间的
    temp_findall_adjust = temp_findall.replace('>','').replace('<','') # 清理>与<
    title.append(temp_findall_adjust)
title

['项目名称', '项目编号', '承担单位', '负责人', '开始年份', '结束年份', '计划类别']

In [8]:
# 提取tag.content
content = []
for i in range(len(tag_content)):
    temp = str(tag_content[i])
    temp_findall = re.findall('>.+<',temp)[0] # 匹配>与<之间的
    temp_findall_adjust = temp_findall.replace('>','').replace('<','') # 清理>与<
    content.append(temp_findall_adjust)
content

['锂离子电池回收技术与设备', '2021GXLH-Z-070', '西安交通大学', '杨国锐', '2021', '2023', '重点项目']

In [11]:
# 整理成 DataFrame 格式
df_temp = pd.DataFrame(content,index=title).T
df_temp

,项目名称,项目编号,承担单位,负责人,开始年份,结束年份,计划类别
0,锂离子电池回收技术与设备,2021GXLH-Z-070,西安交通大学,杨国锐,2021,2023,重点项目


## requests.get

In [3]:
# 加载库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import random
import time
import requests
import urllib.request
import json
import math
import warnings

from urllib.request import urlopen, Request, ProxyHandler, build_opener, HTTPHandler
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from lxml import etree

### requests.get 爬取知乎问答

In [16]:
# 设置链接
url = 'https://www.zhihu.com/api/v5.1/topics/19564496/feeds/essence?offset=10&limit=10&'

# 设置 headers
headers = {
    'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
    'Cookie': '_zap=00ee3f62-9638-4d8f-a9c4-720949695efc; d_c0=AuBW8u9OnhWPTj9FuVG60VdHjEbFP5YZvFo=|1664181146; __snaker__id=i7UfeIcd8E6Z0t0m; _9755xjdesxxd_=32; YD00517437729195%3AWM_TID=kXGcfWCoVddEAFFUVVN6L1JaPSqigAgW; YD00517437729195%3AWM_NI=IqRURZwLNIdh%2Frn3IpHEeV3pmIM8NMWiiGq6FZB0yyRNKDHO8W0KpXk3lYF5o8rDE00BaeUAV%2Fvr1FM4zOlFhjxiwQesJWwpBCoz1%2FAHtkepO8t5DfmO%2BUBb8yTbzaBjTk8%3D; YD00517437729195%3AWM_NIKE=9ca17ae2e6ffcda170e2e6ee92db33f58dba8ec64d91968bb2c15e828e8f82c850adbba9a7b6619a918385f42af0fea7c3b92a8baf9eb6d152b59482aae764edaea2d4cc7c8c9babb7b27caf9783d9cd4795b7a7a8f43e86868ed1fc73929285a5cb45fc8900acf354a9eda4bac53fade8bea5c83cf49eaf86d84b96f1ba93b44e8fecb6b0ef4eacb6b88af65ba7ad86b8f67c829da19bb4258398bc8ce87c8de8bcd6c76eaf91e5b6c16afca79c83f96b82b69c8cbb37e2a3; q_c1=ca7e9a58a85a4c0aaf465412f8a8386a|1664771579000|1664771579000; captcha_session_v2=2|1:0|10:1664782837|18:captcha_session_v2|88:aHVEMmlqODUwUno3TU9uYnZTMGxReGU2NTQrNVFzS0kwa3BLdWxVeE5MKytHaEJ4cGJENTduYmR1alNOSFZFVg==|16dbdd4ad0cdaed9b7ba670441385a8bddbb94960699ee866957038b01bba778; z_c0=2|1:0|10:1664782855|4:z_c0|92:Mi4xQ3JVVEF3QUFBQUFDNEZieTcwNmVGUmNBQUFCZ0FsVk5CLUFuWkFDNHRFLVgtdl9tVFBhU0sxUWUxOFUwXzR3UkFB|51bc2b3a0d030bf38b09621ec45857bcbebdae6c81727ba2fd6c39ee8908a08d; tst=r; gdxidpyhxdE=mvLPJSuIas71JJx254sHoeCc1Muoo%5CzIjqQqJ2pZd2HWr2s8DqNwJuIwCp9w1W7BH096qlXpeM5Q%5CXLxp7%2F1cv%5CPxlg3wl82w%2F4koSR2mdDqC%2BESDccW2jf89tl720A7YIHpo1tZ%2Fub2L84ZJ%2Fuhue%2FIXTcvHMKxZj9%2F6drKhr%2FdJyAA%3A1664789391721; _xsrf=6112f093-2320-46c3-a39e-cd2f6b2065c4; Hm_lvt_98beee57fd2ef70ccdd5ca52b9740c49=1664255094,1664525291,1664771562,1664861025; NOT_UNREGISTER_WAITING=1; __utma=51854390.1068448379.1664861057.1664861057.1664861057.1; __utmc=51854390; __utmz=51854390.1664861057.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utmv=51854390.100--|2=registration_date=20160526=1^3=entry_date=20160526=1; Hm_lpvt_98beee57fd2ef70ccdd5ca52b9740c49=1664862950; KLBRSID=9d75f80756f65c61b0a50d80b4ca9b13|1664862951|1664861024',
    'x-zse-93': '101_3_3.0',
    'x-zse-96': '2.0_g+JdTbLyB1LxVvWW=mjzProocIzPHlNd0lUof7lUMr8nnho7JrWvqfvjRqM5v9vV',
    'x-zst-81': '3_2.0aR_sn77yn6O92wOB8hPZnQr0EMYxc4f18wNBUgpTQ6nxERFZsRY0-4Lm-h3_tufIwJS8gcxTgJS_AuPZNcXCTwxI78YxEM20s4PGDwN8gGcYAupMWufIeQuK7AFpS6O1vukyQ_R0rRnsyukMGvxBEqeCiRnxEL2ZZrxmDucmqhPXnXFMTAoTF6RhRuLPFXxCuCcmwug819p9O9FKPCFMHJCYk93YtJSBDqtYeLHY8qCm_w21YC31eqoTv_2OcwV_6g2BahgKNJXKLDXGbBV_1ggMcRFLEbNfkqCLFueC6XLK9hwGJceGHwH0OgL_OC3_lqO9r_O9ow2feGoMpvHGuuHO6C2sxCg_pckG2v3_CutmOqfz1XVGeJp1bix_Pg38JUxCIhcLc9Xs2hVZuhHmWuNmLCcKLGwyhqgLYhe9qqtBCh2CFCCCr9N1FCHOyBHKsXxG2TrCzqo1LgC0mAN9CbLmsDVYChXCuwFC',
}

# 进行访问
res = requests.get(url, headers=headers)

# 整理结果
# json.loads()可将字符串转为字典，如 json.loads('{a: 2}') -> {a: 2}
ret = json.loads(res.text)

# 查看结果
ret

{'paging': {'totals': 2436732,
  'is_start': False,
  'is_end': False,
  'previous': 'https://www.zhihu.com/api/v5.1/topics/19564496/feeds/essence?offset=0&limit=10&',
  'next': 'https://www.zhihu.com/api/v5.1/topics/19564496/feeds/essence?offset=20&limit=10&'},
 'version': '2.0',
 'data': [{'attached_info': 'CjkQBBoJNTIxNDM5NTMyILr69JkGKPoBMA5AAFIHX2Fuc3dlcloINDQ5MTgzMDRyCjI3MDI0NDUzMDA=',
   'type': 'topic_feed',
   'target_description': '',
   'target': {'is_labeled': False,
    'from_source': '',
    'excerpt': '异地，跟男朋友打语音。不记得因为什么闹了点小矛盾让他伤心了 我：我很喜欢你哦，非常喜欢你，你不要伤心了，我可想跟你羞羞了（诚恳脸 他：别说了别说了，那里要”变长“了 我在电话这头爆笑 ~他是我的初恋，之前都不知道男生这么敏感，觉得他可爱晕了',
    'created_time': 1664957754,
    'id': 2702445300,
    'token': 2702445300,
    'topic_thumbnails': [],
    'author': {'id': '0',
     'url': '',
     'name': '匿名用户',
     'gender': 1,
     'headline': '',
     'url_token': '',
     'avatar_url_template': '',
     'user_type': 'people',
     'is_advertiser': False,
     'avatar_url': 'https://pic4.zh

In [17]:
# 小测试
# 批量获取 20 次爬取结果
for i in range(20):
    res = requests.get(url, headers=headers)
    time.sleep(3)
    ret = json.loads(res.text)
    for j in range(len(ret.get('data'))):
        if ret.get('data')[j].get('target').get('question') != None:
            print('['+ret.get('data')[j].get('target').get('question').get('title')+']')

[单身久了突然有了女票是什么感觉？]
[男生对女朋友的占有欲都很强吗？]
[有一个很作的女朋友是什么感受？]
[男方父母知道自己儿子女朋友怀孕了，会是什么反应?]
[想和女朋友玩密室逃脱，需要注意些什么？]
[请问广大男生，你们跟女朋友聊天的时候会有生理反应吗？]
[王思聪为什么从没有985大学的女朋友？]
[有哪一个瞬间让你对女朋友彻底失望?]
[女朋友看到抖音上的“十根烤肠”，也想要，我说给你带两根，结果就生气了，说我不懂仪式感，可我觉得浪费?]
[男方父母知道自己儿子女朋友怀孕了，会是什么反应?]
[女朋友痛经，很严重，有什么办法可以缓解她的痛苦呢，急？]
[女朋友有个谈了四年的前任，她第一次也是给了她前任。每每想到这个心里就很不舒服，越想越痛苦，要分么？]
[女朋友的过去有点复杂我该怎么办？]
[王思聪为什么从没有985大学的女朋友？]
[如何评价王宝强的新女友？]
[有哪一个瞬间让你对女朋友彻底失望?]
[你们的女朋友是怎么样的？]
[第一次带对象回家是一种什么样的体验？]
[为什么给女朋友花钱不舒服？]
[女朋友是医生是种什么体验?]
[男博士是否不愿意找学历太高的女朋友，比如女硕士或是女博士？]
[有哪一个瞬间让你对女朋友彻底失望?]
[女朋友要18万彩礼，我该怎么办?]
[找女朋友是不是要趁早（卑微大学生）？出来工作以后是不是比较难找到女朋友？]
[有哪些事是男生很想让女朋友知道的？]
[男博士是否不愿意找学历太高的女朋友，比如女硕士或是女博士？]
[为什么很多很好的男孩子没有女朋友甚至没人追？]
[把自己女朋友公布在朋友圈真的很难吗？]
[有一个很乖的女朋友是一种怎样的体验?]
[女朋友看到抖音上的“十根烤肠”，也想要，我说给你带两根，结果就生气了，说我不懂仪式感，可我觉得浪费?]
[找个越南女朋友是什么体验？]
[女朋友要求我来回2000公里开车去接她两个堂妹合理吗？]
[有个可爱的女朋友是什么感觉？]
[女朋友睡着了，我半夜在王者里随便拉了一个女生玩了两把游戏，这算花心吗？]
[有哪一个瞬间让你对女朋友彻底失望?]
[女朋友不愿意和自己打王者，而去和她的闺蜜玩是为什么？]
[有一个爱玩游戏的女朋友是种什么体验？]
[为什么女生在约会的时候喜欢迟到？]
[有哪一个瞬间让你对女朋友彻底失望?]
[女朋友不愿意和自己打王者，而去和她的闺蜜玩是为什么？

### requests.get 爬取百度地图位置信息

In [1]:
# 在进行 requests.get 爬取时，可以用 requests.get 中的参数 params 传递 data

In [8]:
# 源网页
# 'https://map.baidu.com/search/%E5%B1%B1%E4%B8%9C%E5%A4%A7%E5%AD%A6/@13030166.298247589,4364635.8100000005,13.73z?querytype=s&da_src=shareurl&wd=%E5%B1%B1%E4%B8%9C%E5%A4%A7%E5%AD%A6&c=289&src=0&pn=0&sug=0&l=13&b=(13495175,3623687;13544327,3646791)&from=webmap&biz_forward=%7B%22scaler%22:2,%22styles%22:%22pl%22%7D&device_ratio=2'
# 目标--获取地点所在省(市)

In [226]:
import math

In [219]:
df = pd.read_excel('./20230915发展研究中心.xlsx', sheet_name='Sheet4')

In [242]:
lst = df['名单2'].to_list()
lst = [v for v in lst if not pd.isna(v)]

In [246]:
# 设置名单1
# name_1 = pd.DataFrame([re.sub('\d+','',v).replace('. ','') for v in lst], columns = ['公司名称'])['公司名称'].to_list()
# 设置名单2
name_2 = pd.DataFrame([re.sub('\d+','',v).replace('. ','') for v in lst], columns = ['公司名称'])['公司名称'].to_list()

In [244]:
lst_city = []

In [162]:
for i in range(len(name_2)):
    # 测试 '山东大学'
    # requests
    # 设置链接
    url = 'https://map.baidu.com'

    # 设置 headers
    headers = {
        'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.37',
        'Content-Encoding': 'gzip',
        'Content-Security-Policy-Report-Only': "default-src https: 'unsafe-inline' 'unsafe-eval' data: blob: ; report-uri https://reports.baidu.com/csp-report/map-web",
        'Content-Type': 'text/javascript;charset=UTF-8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6,ja;q=0.5'
    }

    # 设置 params 参数
    data={
        'qt': 's',
        'wd': lst_cor[i]
    }

    # 进行爬取
    res = requests.get(url=url, params=data, headers=headers)

    # 替换字符，进行转码
    res_text = res.text.replace("/","\\")
    res = res_text.encode('utf-8').decode('unicode-escape')  # 对 \\xa0 类编码字符进行转码



    '''
    # 获取机构所在省、自治区、直辖市
    tag_prov = re.findall('"prov_name":".+"',res)
    tag_city = re.findall('"city_name":".+"',res)
    if tag_prov != []:                           # 匹配省份
        tag = tag_prov
        prov = tag[0].split('"')[3]
    elif tag_city != []:                         # 匹配直辖市
        tag = tag_city
        prov = tag[0].split('"')[3]
    else:
        prov = 'NONE'                            # 匹配失败
    '''

    check = re.findall('"api_admin_info":{(.+?)}',res)
    if check != []:
        temp_city = '{'+re.findall('"api_admin_info":{(.+?)}',res)[0]+'}'
        city = json.loads(temp_city).get('city_name')

    else:
        city = 'NONE'

    lst_city.append(city)

    time.sleep(1)
    
    print(str(i+1)+'/'+str(len(lst_cor)), end='\r')

In [247]:
lst_city = []

In [248]:
for i in range(len(name_2)):
    
    # 测试 '山东大学'
    # requests
    # 设置链接
    url = 'https://map.baidu.com'

    # 设置 headers
    headers = {
        'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.37',
        'Content-Encoding': 'gzip',
        'Content-Security-Policy-Report-Only': "default-src https: 'unsafe-inline' 'unsafe-eval' data: blob: ; report-uri https://reports.baidu.com/csp-report/map-web",
        'Content-Type': 'text/javascript;charset=UTF-8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6,ja;q=0.5'
    }

    # 设置 params 参数
    data={
        'qt': 's',
        'wd': name_2[i]
    }

    # 进行爬取
    res = requests.get(url=url, params=data, headers=headers)

    # 替换字符，进行转码
    res_text = res.text.replace("/","\\")
    res = res_text.encode('utf-8').decode('unicode-escape')  # 对 \\xa0 类编码字符进行转码

    # temp_city = '{'+re.findall('"api_admin_info":{(.+?)}',res)[0]+'}'
    # city = json.loads(temp_city).get('city_name')
    temp_city = '{'+re.findall('"current_city":{(.+?)}',res)[0]+'}'
    city = json.loads(temp_city).get('name')
    
    lst_city.append(city)

    time.sleep(1)
    
    print(str(i+1)+'/'+str(len(name_2)), end='\r')

In [249]:
pd.DataFrame(lst_city).to_excel('./name_2.xlsx', index=False)

In [ ]:
'''
# 获取机构所在省、自治区、直辖市
tag_prov = re.findall('"prov_name":".+"',res)
tag_city = re.findall('"city_name":".+"',res)
if tag_prov != []:                           # 匹配省份
    tag = tag_prov
    prov = tag[0].split('"')[3]
elif tag_city != []:                         # 匹配直辖市
    tag = tag_city
    prov = tag[0].split('"')[3]
else:
    prov = 'NONE'                            # 匹配失败
'''

check = re.findall('"api_admin_info":{(.+?)}',res)
if check != []:
    temp_city = '{'+re.findall('"api_admin_info":{(.+?)}',res)[0]+'}'
    city = json.loads(temp_city).get('city_name')

else:
    city = 'NONE'

lst_city.append(city)

time.sleep(1)

In [156]:
city

'长沙市'

In [145]:
temp_a = '{'+re.findall('"api_admin_info":{(.+?)}',res)[0]+'}'
# temp_a = '{'+re.findall('"current_city":{(.+?)}',res)[0]+'}'

json.loads(temp_a).get('city_name')

'西安市'

## requests.post

In [2]:
# 当请求为 post 时，需要利用 requests.post 中的 data 参数进行传递 (类似 requests.get 的 params)

In [1]:
# 加载库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import random
import time
import requests
import urllib.request
import json
import math

from urllib.request import urlopen, Request, ProxyHandler, build_opener, HTTPHandler
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from lxml import etree

### requests.post 爬取导师信息

In [2]:
# 源网页--'https://www.scholarmate.com/inspage/institution/main?des3InspageId=5QMOHWK86Lg%3D&model=homepage'
# 目标--爬取全部学者信息

In [7]:
# 设置参数
org = '华东师范大学'

In [8]:
# Step 1
# 获取负载 data 中的 insID
# get_insID
# param: org---机构名称
# org = '华东师范大学'

def get_insID(org):
    
    url = 'https://www.scholarmate.com/inspage/outside/institution/ajaxshowinslist'  # 多了个 outside
    
    # 生成 data
    data={
        'page.totalCount': -1,
        'page.pageNo': 1,
        'currentLoad': 0,
        'searchKey': org,
        'searchType': 1,
        'item': 2,
    }
    
    # 爬取网页
    response = requests.post(url=url, data=data)
    id_all = re.findall('src=".+"', response.text)
    insID = []
    for i in range(len(id_all)):
        insID_i = re.findall('\d+', re.findall('src=".+"',response.text)[i])[1]
        insID.append(insID_i)
    return insID

In [9]:
# 测试函数 get_insID
insID = get_insID(org)
insID

['771']

In [ ]:
# Step 2
# 从负载 payload 导入 data
# 'page.totalCount':     导师个数
# 'page.pageNo':         当前页码
# 'insId':               机构代码
# 'queryFields.orderBy'  排序方式

# 最后一关 --- 'page.totalCount' 怎么找？'page.pageNo' 怎么循环？
# 已解决(20220908 Thu 15:02)

# page 1 --- 获取 page.totalCount
# 设置链接
url = 'https://www.scholarmate.com/inspage/institution/ajaxscholarlist'

# 导入 data
# 以第一个 insID 为主，即 insID[0]
data = {'page.totalCount': -1,
        'page.pageNo': 1,
        'insId': insID[0],
        'queryFields.orderBy': 'pubCount'}

# 进行爬取
response = requests.post(url=url, data=data)
# 提取文本
text = response.text
# 获取 page.totalCount
page_totalCount = re.findall('\d+',re.findall('smate_count=".+"',text)[0])[0]
# 获取 page.pageNo 最大迭代值 page_max
page_max = math.ceil(np.array([page_totalCount]).astype(np.float64)[0]/10)

# Scholar-Mate 导师链接全量获取
data_all = pd.DataFrame(None,columns=['pi_name','pi_org','href'])
for i in range(page_max):
    # page 1 ~ page page_max --- 代入 page_totalCount
    # 导入 data
    page_num = i+1
    data = {'page.totalCount': page_totalCount,
            'page.pageNo': page_num,
            'insId': insID[0],
            'queryFields.orderBy': 'pubCount'}
    # 进行爬取
    response = requests.post(url=url, data=data)
    # 提取文本
    text = response.text

    # 获取导师链接
    href_all = re.findall('href=".+"',text)
    # 对链接进行去重
    href_temp = []
    for h in href_all:
        if h not in href_temp:
            href_temp.append(h)
    # 获取最终链接
    href = []
    for v in href_temp:
        href_v = re.findall('".+"',v)[0].split(' ')[0].replace('"','')
        href.append(href_v)

    # 获取导师机构
    pi_org = [org]*len(href)

    # 获取导师姓名
    soup = BeautifulSoup(text, "html.parser")
    pi_name = []
    for f in href:
        pi_name_temp = soup.find_all('a',href=f)[1].get_text()
        pi_name.append(pi_name_temp)
    data_temp = []
    for v in pi_name, pi_org, href:
        data_temp.append(v)
    data_temp = pd.DataFrame(data_temp,index=['pi_name','pi_org','href']).T
    data_all = pd.concat([data_all,data_temp], ignore_index=True)
    print(org+' 第 '+str(page_num)+'/'+str(page_max)+' 页爬取完成',end='\r')
    if page_num == page_max:
        data_all.to_excel('./'+org+'.xlsx')
        data_all = pd.DataFrame(None,columns=['pi_name','pi_org','href'])
    time.sleep(0.5)